In [1]:
# Cleaned up notebook with to read data from kml file and convert it to a pandas dataframe

# to read correctly the kml file should be organized as foolows
    # <Document><Folder> should contain folders of each individual field
    # each field should contain just two layers, both polygons
    # the first polygon should be the Foul ground polygon
    # the second polygon should be the field (FOP) polygon



In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\My Places_dec_9_22.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])

In [2]:
## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Clean up the data

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]


## remove any numberic characters and . from field names
df['field'] = df['field'].str.replace(r'\d+', '')
    

C:\Users\Justin\AppData\Local\Temp\ipykernel_16344\3425283714.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16344\3425283714.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16344\3425283714.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16344\3425283714.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [3]:
## Create a (lat, long) pair for home plate 
df['home_plate'] = df['foul'].str.split(' ').str[0]

## Apply lambda function to format polygon coordinates and pass to the area function
# #OPutput in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)

# drop the square meter area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

# calculate the ratio of foul ground to total area
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100

## Convert the coordinates to a list of tuples
df['foul'] = df['foul'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])
df['fop'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])



In [6]:
### Create shapely polygons from the coordinates

from shapely.geometry import Polygon
import geopandas as gpd

df['foul'] = df['foul'].apply(lambda x: Polygon(x))
df['fop'] = df['fop'].apply(lambda x: Polygon(x))


df['field'].unique()

array(['Beal City HS', 'Buchanan HS',
       'Grosse Pointe Woods University Liggett',
       'Jackson Lumen Christi HS', 'Kalamazoo Christian',
       'Shrine HS Royal Oak', 'Hemlock HS', 'Chesaning Union HS',
       'Glen Lake Community School', 'Bullock Creek HS - Midland',
       'Rogers City HS', 'Standish-Sterling HS', 'New Lothrop HS',
       'Rochester Hills Lutheran Northwest', 'Bridgman HS',
       'Indian River Inland Lakes HS', 'Breckenridge HS',
       'Essexville Garber HS', 'Watervliet HS', '. Bad Axe HS',
       'Cheboygan HS', 'Marine City Cardinal Mooney HS', 'Adrian College',
       'Aldai Stevenson High School',
       'Alden - Antrim County - Lakeside Field', 'Algonac High School',
       'Allen Park High School', 'Athens High School - Troy',
       'Bangor High School', 'Bath High School', 'Battle Creek (Bailey)',
       'Battle Creek lakeview', 'Bay City All Saints HS',
       'Bay City All Saints Youth Field', 'Bay City john Glen',
       'Blissfield', 'Boeve Ba

# Working to this point geometries all load.
## export file below to send to tableau

In [6]:
### Export files to csv and geojson

### Having issues with exporting into the data folder

df_export = df.drop(['foul', 'fop'], axis=1)
df_export.to_csv('fields_notebook_export_w_geos.csv', index=True)
# df.to_json('data\fields.geojson', orient='records')

In [7]:
# https://gis.stackexchange.com/questions/294206/how-to-create-a-simple-polygon-from-coordinates-in-geopandas-with-python

# from shapely import wkt

# import geopandas as gpd
# from shapely.geometry import Polygon

# lon_lat_list = df['foul'][3]
# polygon_geom = Polygon(lon_lat_list)
# polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])       

# import folium
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m
# df['foul_shapely'] = df['foul_str'].apply(lambda x: wkt.loads(x)) # only works with strings
# df.head()

Working up to export. cell above is example code for plotting purposes
